In [1]:
import requests
import pickle
from sklearn.preprocessing import StandardScaler
import time


In [2]:
# URLs for Blynk sensor data
BLYNK_URLS = {
    "V1": "https://blynk.cloud/external/api/get?token=a0R-kWpg-zJ6EX7r0U67MfkoKd72XTwK&V0",  # X accelerometer
    "V2": "https://blynk.cloud/external/api/get?token=a0R-kWpg-zJ6EX7r0U67MfkoKd72XTwK&V1",  # Y accelerometer
    "V3": "https://blynk.cloud/external/api/get?token=a0R-kWpg-zJ6EX7r0U67MfkoKd72XTwK&V2",  # Z accelerometer
    "V4": "https://blynk.cloud/external/api/get?token=a0R-kWpg-zJ6EX7r0U67MfkoKd72XTwK&V3"   # Vibration sensor
}


In [3]:
# Function to get sensor data from Blynk
def get_blynk_data(pin):
    url = BLYNK_URLS.get(pin)
    try:
        # Make the HTTP GET request
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        data = response.json()  # Parse JSON response
        return float(data)    # Return the sensor value as a float
    except (requests.RequestException, ValueError) as e:
        print(f"Error fetching data from Blynk for pin {pin}: {e}")
        return None

In [4]:
# Fetch sensor data from Blynk virtual pins
accel_x = get_blynk_data("V1")  # X-axis accelerometer
accel_y = get_blynk_data("V2")  # Y-axis accelerometer
accel_z = get_blynk_data("V3")  # Z-axis accelerometer
vibration = get_blynk_data("V4")  # Vibration sensor

# Check if all data was fetched successfully
if None in [accel_x, accel_y, accel_z, vibration]:
    print("Error fetching sensor data.")
else:
    # Print the sensor values
    print(f"Accel X: {accel_x}, Accel Y: {accel_y}, Accel Z: {accel_z}, Vibration: {vibration}")


Accel X: 1.0, Accel Y: 2.0, Accel Z: 1.0, Vibration: 1.0


In [5]:
# Function to load the pretrained model and scaler
def load_model():
    try:
        with open('rf_model.pkl', 'rb') as file:
            model = pickle.load(file)
        with open('scaler.pkl', 'rb') as file:
            scaler = pickle.load(file)
        return model, scaler
    except FileNotFoundError as e:
        print(f"Error loading model or scaler: {e}")
        return None, None

In [6]:
# Function to predict landslide using sensor data
def predict_landslide(sensor_data, model, scaler):
    try:
        # Scale the sensor data
        sensor_data_scaled = scaler.transform([sensor_data])
        
        # Predict landslide (1) or no landslide (0)
        prediction = model.predict(sensor_data_scaled)
        
        return "Landslide Detected" if prediction == 1 else "No Landslide Detected"
    except Exception as e:
        print(f"Error during prediction: {e}")
        return "Prediction Error"


In [7]:
# Function to send notification to Blynk
def send_blynk_notification(message):
    url = f"https://blynk.cloud/external/api/notify?token=a0R-kWpg-zJ6EX7r0U67MfkoKd72XTwK&msg={message}"
    try:
        requests.get(url)
    except requests.RequestException as e:
        print(f"Error sending notification: {e}")

In [8]:
# Load model and scaler
model, scaler = load_model()
if model is None or scaler is None:
    print("Failed to load model or scaler. Exiting.")
    exit()

In [10]:
 # Create sensor data array for prediction
sensor_data = [accel_x, accel_y, accel_z, vibration]

# Predict if there's a landslide
prediction = predict_landslide(sensor_data, model, scaler)
print("Prediction:", prediction)

# Send a notification if a landslide is detected
if prediction == "Landslide Detected":
    send_blynk_notification("Warning! Landslide Detected!")

# Wait for 5 seconds before the next check
time.sleep(5)

D:\anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Prediction: Landslide Detected
